In [ ]:
import tensorflow_hub as hub
import cv2
import keras
import numpy
import tensorflow as tf
import pandas as pd
import numpy as np

# Loading models
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
model = keras.models.load_model("dog-detector-model")
labels = pd.read_csv("labels.csv",sep=";",index_col="ID")
labels = labels["OBJECT (2017 REL.)"]

width = 512
height = 512

doggo_labels = np.array(pd.read_csv("doggo-labels.csv")["breed"])
unique_breeds = np.unique(doggo_labels)

def get_pred_label(prediction_probabilities):
    return unique_breeds[np.argmax(prediction_probabilities)]

def process_image(frame):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize(frame, size=[224, 224])

    return frame

def batch(frame):
    data = tf.data.Dataset.from_tensor_slices([np.asarray(frame)])
    data_batch = data.map(process_image).batch(1)    

    return data_batch


In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    inp = cv2.resize(frame, (width, height ))
    rgb_tensor = tf.convert_to_tensor(inp, dtype=tf.uint8)

    #Add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor , 0)
    
    boxes, scores, classes, num_detections = detector(rgb_tensor)
    
    pred_labels = classes.numpy().astype("int")[0]
    
    pred_labels = [labels[i] for i in pred_labels]
    pred_boxes = boxes.numpy()[0].astype("int")
    pred_scores = scores.numpy()[0]
    fps = int(cap.get(cv2.CAP_PROP_FPS))
   
    for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < 0.2:
            continue
            
        font = cv2.FONT_HERSHEY_SIMPLEX
        
        if label == "dog":
            dog_label = get_pred_label(model.predict(batch(inp[ymin:ymax-10, xmin:xmax-10])))
            rep = dog_label.replace("_", " ")
            cap_rep = rep.capitalize()
            
            score_txt = f'{100 * round(score, 0)}'
            inp = cv2.rectangle(inp ,(xmin, ymax),(xmax, ymin),(0,255,0),1) 
            cv2.putText(inp,dog_label,(xmin, ymax-10), font, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(inp,score_txt,(xmax, ymax-10), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        print(fps)
            
            
    cv2.imshow("window",inp)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()